In [ ]:
!pip install qiskit 

from IPython.display import clear_output
clear_output(wait=False)

In [ ]:
#Includes true purity and faster qiskit implementation

from qiskit import *
from qiskit.visualization import plot_histogram, plot_state_qsphere
from qiskit.extensions import UnitaryGate, Initialize
from scipy.linalg import expm
import numpy as np
import random
import matplotlib.pyplot as plt
from scipy.optimize import differential_evolution
from datetime import datetime
from qiskit.quantum_info import purity

In [ ]:
start_time = datetime.now()
simulator = Aer.get_backend('qasm_simulator')
svsim = Aer.get_backend('statevector_simulator')

H = [[0.25, 0.24],                                  #Operator whose eigenvalues and eigenvectors are to be found 
     [0.24, 0.25]] 

H = np.matrix(H)

U = expm(-1j * H)
Ugate = UnitaryGate(U, label = "U = exp(-iHt)")
CUgate = Ugate.control(1)

cos = np.cos
sin = np.sin
pi = np.pi
log = np.log
exp = np.exp
sqrt = np.sqrt
dot = np.dot


def prob(outcome: int, bv_s: list, bv_m: list) -> float:
    tcm: int = outcome
    pr: float = (1. / 2.) * (1 + (-1) ** tcm * dot(bv_s, bv_m))
    return pr


def bloch_vector_pure(theta: float, phi: float) -> list:
    rx: float = sin(pi * theta) * cos(pi * phi)
    ry: float = sin(pi * theta) * sin(pi * phi)
    rz: float = cos(pi * theta)
    bv: list = [rx, ry, rz]
    return bv


def bv_state(theta_psi: float, phi_psi: float, ham: list) -> list:
    # bloch vector of Hamiltonian
    e0: float = ham[0]
    ev: list = [ham[1], ham[2], ham[3]]
    e1: float = sqrt(dot(ev, ev))
    bv_e: list = [x / e1 for x in ev]
    if dot(bv_e, bv_e) != 1:
        print("bloch vector of hamiltonian is not normalizable")
        exit(1)
    # check if eig values of Hamiltonian are the required range

    if 0 <= e0 + e1 < 1. / 2. and 0 <= e0 - e1 < 1. / 2.:
        pass
    else:
        print("hamiltonian is out of range for requirement. 0 <= eig(H) < pi/2")
        exit(1)

    # print(ev, e1, bv_e)
    # bloch vector of initial state psi
    bv_psi: list = bloch_vector_pure(theta_psi, phi_psi)
    # inner product between bv_e and bv_psi
    ov_ep: np.ndarray = dot(bv_e, bv_psi)
    # bloch vectors of control-qubit state
    rx: float = cos(pi * e0) * cos(pi * e1) - ov_ep * sin(pi * e0) * sin(pi * e1)
    ry: float = - sin(pi * e0) * cos(pi * e1) - ov_ep * cos(pi * e0) * sin(pi * e1)
    rz: float = 0.
    bv_s: list = [rx, ry, rz]
    return bv_s


def free_energy_model(coefficients_a: list, theta_psi: float, phi_psi: float) -> float:
    a: list = coefficients_a
    fem: float = a[0] + a[1] * cos(phi_psi) + a[2] * sin(phi_psi)
    return fem


def quantum_measurement(param_theta, param_phi, meas_theta, meas_phi, m, ss) : 
    
    qc = QuantumCircuit(2,2)
    qc.h(0)
    qc.u(param_theta * pi , param_phi * pi , 0, 1)
    qc.append(CUgate, [0,1])

    qc.u(-meas_theta * pi, 0,-meas_phi * pi, 0 )
    
    if m == ss - 1 :
      state = execute(qc, backend = svsim).result().get_statevector()
      state_purity = purity(state, validate = True) 
      print("----State Purity---- =", state_purity)

    qc.measure(0,0) 
    result = execute(qc, backend = simulator, shots = 1).result()
    result_dict = result.get_counts()
    measurement_output = int(list(result_dict.keys())[list(result_dict.values()).index(1)])
    #print(measurement_output)
    return measurement_output


def cost_free_energy(coefficients_a: list, list_theta_phi_free_energy: list) -> float:
    ca: list = coefficients_a
    ltpfe: list = list_theta_phi_free_energy
    # cost_free_energy
    cfe: float = 0.0
    # length of latest data to employ
    nldtc: int = __num_latest_data_to_count
    for el in ltpfe[-nldtc:]:
        theta_psi: float = el[0]
        phi_psi: float = el[1]
        feo: float = el[2]
        fem: float = free_energy_model(ca, theta_psi, phi_psi)
        cfe += (fem - feo) ** 2
    return cfe / nldtc


def kullback_leibler_divergence(list_purity_energy_est: list, learning_data: list) -> float:
    purity_est: float = list_purity_energy_est[0]
    energy_est: float = list_purity_energy_est[1]
    tcm_list: list = learning_data[0]
    theta_psi: float = learning_data[1]
    phi_psi: float = learning_data[2]
    ham: list = learning_data[3]

    lld: float = len(tcm_list)
    n1: float = 0.
    for tcm in tcm_list:
        n1 += tcm
    freq: list = [(lld - n1) / lld, n1 / lld]

    bv_s: list = bv_state(theta_psi, phi_psi, ham)
    bv_measurement_est: list = bloch_vector_pure(0.5, -energy_est)
    bv_me: list = [x * purity_est for x in bv_measurement_est]
    # bv_me = purity_est * bv_measurement_est

    pr0: float = prob(0, bv_s, bv_me)
    pr1: float = 1 - pr0
    pr: list = [pr0, pr1]

    kld = 0.0
    if freq[0] > 0.:
        kld += freq[0] * log(freq[0])
    elif freq[0] < 0.:
        print("negative frequency(0)??")
    if pr[0] > 0.:
        kld -= freq[0] * log(pr[0])
    elif pr[0] < 0.:
        print("negative probability(0)??")
    if freq[1] > 0.:
        kld += freq[1] * log(freq[1])
    elif freq[1] < 0.:
        print("negative frequency(1)??")
    if pr[1] > 0.:
        kld -= freq[1] * log(pr[1])
    elif pr[1] < 0.:
        print("negative probability(1)??")

    # print(kld)
    return kld


def estimate_purity_energy(sample_size: int, s_param: list, m_param: list) -> list:
    num_success_events: int = 0
    ss: int = sample_size
    theta_psi: float = s_param[0]
    phi_psi: float = s_param[1]
    ham: list = s_param[2]
    theta_m: float = m_param[0]
    phi_m: float = m_param[1]
    purity_est: float = m_param[2]
    energy_est: float = m_param[3]

    kld: float = -1.

    b_purity: tuple = (0., 1.)
    b_energy: tuple = (0.0 + 1e-5, 0.5 - 1e-5)

    m: int
    tcm: int

    learning_data: list
    tcm_list: list = []
    for m in range(ss):
        tcm = quantum_measurement(theta_psi, phi_psi, theta_m, phi_m, m , ss)
        tcm_list.append(tcm)
        learning_data = [tcm_list, theta_psi, phi_psi, ham]  # Add new data to a list
        if tcm == 1:
            new_purity_energy = differential_evolution(kullback_leibler_divergence,
                                                       args=(learning_data,), bounds=[b_purity, b_energy],
                                                       popsize=__ps, mutation=__mt, recombination=__rc)
            if new_purity_energy.success:
                pass
            else:
                print("minimizing kullback_leibler_divergence fails")
            purity_est, energy_est = new_purity_energy.x
            kld = new_purity_energy.fun
            phi_m = -energy_est
        else:
            num_success_events += 1

    success_probability: float = num_success_events / sample_size
    # if success_probability == 1.:
    if success_probability >= 1. - 0.1 / sqrt(sample_size):
        print("The number of success events reach the sample_size.")
    # print("purity_est =", purity_est, "energy_est =", energy_est, "phi_m =", phi_m)
    return [purity_est, energy_est, phi_m, success_probability, kld]


def free_energy(purity: float, energy: float) -> float:
    return energy - __temperature * purity


def find_state_phi(phi_psi: float, args: list) -> float:
    ca: list = args[0]
    theta_psi: float = args[1]
    fsp: float = free_energy_model(ca, theta_psi, phi_psi)
    return fsp


def estimate_minimal_free_energy(n_iterations: int, sample_size: int, theta_psi: float, ham: list) -> list:
    s_param: list
    m_param: list
    purity_est: float = 0.
    energy_est: float = 1.
    f_energy: float = -1.
    ps: float = -1.
    kld: float = -1.
    list_theta_phi_free_energy: list = []
    ltpf: list = list_theta_phi_free_energy

    ca: list = []
    bbnd: tuple = (-3.0, 3.0)
    b_ca: list = [bbnd] * 3
    cfe: float = 0.

    # theta_psi: float = random.random()
    # theta_psi: float = 1./2.
    # phi_psi ranges between -1 and 1
    phi_psi: float = (random.random() * 2 - 1.)
    # phi_psi: float = 1.
    # phi_m ranges between -1 and 1
    # phi_m: float = (random.random() * 2 - 1.)
    phi_m: float = 0.
    theta_m: float = 1. / 2.
    argmts: list
    cfe_fn: float = 0.

    nit: int
    for nit in range(n_iterations):
        # Finding energy
        s_param = [theta_psi, phi_psi, ham]
        m_param = [theta_m, phi_m, purity_est, energy_est]

        purity_est, energy_est, phi_m, ps, kld = estimate_purity_energy(sample_size, s_param, m_param)
        f_energy = free_energy(purity_est, energy_est)
        print("-------------------------------------------------------------------------")
        print(nit, "-th: prob_success =", ps)
        print("-------------------------------------------------------------------------")
        
        print("purity_est =", purity_est, "energy_est =", energy_est, "free_energy_est =", f_energy)
        print("theta_psi =", theta_psi, "phi_psi =", phi_psi, "theta_meas =", theta_m, "phi_meas =", phi_m,
              "Kullback-Leibler Divergence =", kld)
        # test if reaching the precision
        if ps > 1. - 0.1 / sqrt(sample_size) and nit > __num_latest_data_to_count:
            break

        # Finding a parameters----------------------------------------------------------------------
        ltpf.append([theta_psi, phi_psi, f_energy])
        lnt_ltpf = len(ltpf)
        if lnt_ltpf < __num_latest_data_to_count:
            phi_psi = (random.random() * 2 - 1)
            phi_m = (random.random() * 2 - 1)
        else:
            # phi_psi = 0.
            ca_old: list = ca
            new_coefficients_a = differential_evolution(cost_free_energy, bounds=b_ca, args=(ltpf,),
                                                        popsize=__ps, mutation=__mt,
                                                        recombination=__rc)  # Prof's suggestion (modified  a bit)
            if not new_coefficients_a.success:
                print("minimizing cost_free_energy fails")
                continue
            else:
                ca = new_coefficients_a.x
                cfe = new_coefficients_a.fun
                print("minimal cost of free energy =", cfe, "param a =", [ca[0], ca[1], ca[2]])

            argmts = [ca, theta_psi]
            new_phi = differential_evolution(find_state_phi, bounds=[(-1, 1)],
                                             args=(argmts,), popsize=__ps,
                                             mutation=__mt, recombination=__rc)
            if not new_phi.success:
                print("find_state_phi fails")
                continue
            else:
                phi_psi = new_phi.x[0]
                cfe_fn = new_phi.fun[0]
                print("phi_psi =", phi_psi)
                print("minimum free energy function in model =", cfe_fn)
    return [theta_psi, phi_psi, purity_est, energy_est, f_energy, ps, cfe, cfe_fn]


__ps: int = 5
__mt: tuple = (0.01, 0.1)
__rc: float = 0.7
__temperature: float = 0.
__num_latest_data_to_count: int = 5


def main():
    ham: list = [0.25, 0.24, 0., 0]  # hamiltonian vector = [h0, hx, hy, hz], coefficients of Pauli operators
    theta_psi: float = 1. / 2

    n_iterations: int = 100
    # initial sample size, being increased
    sample_size: int = 50
    # determine the numerical precision for purity, energy, angles

    estimate_minimal_free_energy(n_iterations, sample_size, theta_psi, ham)


if __name__ == '__main__':
    main()
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

In [ ]:
_start = datetime.now()
_end = datetime.now()
print('Duration: {}'.format(end_time - start_time))